In [1]:
!pip install langchain
!pip install langchain_community
!pip install openai
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 24.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.9.0 requires 

In [2]:
%%writefile qwen.env
## Qwen base_url
OPENAI_BASE_URL='https://dashscope.aliyuncs.com/compatible-mode/v1'
DASHSCOPE_API_KEY='sk-d4c3f773af5d442c97eb7d9e40479c55'
OPENAI_API_KEY=${DASHSCOPE_API_KEY}

Writing qwen.env


In [3]:
## 使用qwen key
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv('qwen.env',override=True)

import os
print(os.environ['OPENAI_BASE_URL'],os.environ['OPENAI_API_KEY'])

https://dashscope.aliyuncs.com/compatible-mode/v1 sk-d4c3f773af5d442c97eb7d9e40479c55


## Chains
链允许我们将多个组件组合在一起，创建一个单一而连贯的应用程序。例如，我们可以创建一个链，接受用户输入，使用 PromptTemplate 进行格式化，然后将格式化的响应传递给 LLM。我们可以通过将多个链组合在一起或将链与其他组件组合来构建更复杂的链。

### 连接llm和prompt_template
直接llm|prompt_template即可

In [28]:
import warnings
warnings.filterwarnings('ignore')

from langchain.chat_models import ChatOpenAI 
from langchain.prompts import ChatPromptTemplate  
#from langchain.chains import LLMChain   

model_name='qwen2-7b-instruct'
llm = ChatOpenAI(model=model_name,temperature=0.0) 

prompt = ChatPromptTemplate.from_template("描述制造{product}的一个公司的最佳名称是什么?")

## 直接连接llm和prompt
chain = prompt|llm 

product = "大号床单套装"
chain.invoke(product)

AIMessage(content='为一家制造大号床单套装的公司选择一个最佳名称时，可以考虑以下几个因素：\n\n1. **品牌定位**：首先明确公司的品牌定位。是专注于高端奢华市场，还是更倾向于平价大众市场？是追求简约现代风格，还是有特定的设计理念或主题？\n\n2. **目标受众**：了解目标顾客群体的需求和喜好。比如，年轻人可能更喜欢时尚、个性化的产品，而家庭用户可能更注重耐用性和舒适性。\n\n3. **独特性与记忆性**：选择一个既有意义又能让人记住的名称。避免使用过于常见或已经广泛使用的名称，以增加品牌在市场中的独特性和辨识度。\n\n4. **文化关联**：如果品牌想要传达某种特定的文化或情感价值，名称中可以融入相关的元素。这有助于建立品牌与消费者之间的情感连接。\n\n基于以上几点，以下是一些示例名称及其背后的考量：\n\n- **云端舒适（CloudyComfort）**：适用于专注于提供高品质、奢华体验的品牌，强调产品带来的舒适感受如同在云端一般。\n- **大空间梦想（BigSpaceDreams）**：针对追求宽敞、舒适睡眠体验的消费者，名称中直接表达了品牌愿景和目标市场。\n- **永恒棉柔（EternalSoftness）**：强调产品的持久耐用性和极佳的触感，适合那些注重长期品质和舒适性的品牌。\n- **夜之羽翼（NightWings）**：采用富有想象力和浪漫色彩的名称，适合追求独特设计和梦幻氛围的品牌，尤其适合年轻消费者群体。\n- **舒适领地（ComfortTerritory）**：简洁有力，易于记忆，适合任何专注于提供全方位舒适体验的品牌，既适用于高端市场也适用于大众市场。\n\n最终选择的名称应该能够准确反映品牌的定位、价值观以及目标市场的偏好，同时具有一定的创新性和吸引力。', response_metadata={'token_usage': {'completion_tokens': 384, 'prompt_tokens': 21, 'total_tokens': 405}, 'model_name': 'qwen2-7b-instruct', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-18daa

In [27]:
## 多组连接可以逐一连接，要注意连接顺序
from langchain.chains import SimpleSequentialChain
from langchain_core.output_parsers import StrOutputParser
llm = ChatOpenAI(model=model_name,temperature=0.9)

# 提示模板 1 ：这个提示将接受产品并返回最佳名称来描述该公司
first_prompt = ChatPromptTemplate.from_template(   
    "描述制造{product}的一个公司的最好的名称是什么"
)
chain_one = first_prompt|llm |StrOutputParser()

# 提示模板 2 ：接受公司名称，然后输出该公司的长为20个单词的描述
second_prompt = ChatPromptTemplate.from_template(   
    "写一个20字的描述对于下面这个\
    公司：{company_name}的"
)
## 逐个连接在一起即可
chain_two = (
    {"company_name": chain_one}|second_prompt|llm|StrOutputParser()
)
product = "大号床单套装"
chain_two.invoke({"product": "大号床单套装"})

'"大号梦想丝柔"（GrandDreamSilky）——结合了大号床的特性与对极致舒适睡眠的追求，同时“丝柔”二字强调产品材质的柔软与品质，易于记忆且富有吸引力。'

In [29]:
## 使用SimpleSequentialChain连接

# 提示模板 1 ：这个提示将接受产品并返回最佳名称来描述该公司
first_prompt = ChatPromptTemplate.from_template(   
    "描述制造{product}的一个公司的最好的名称是什么"
)
chain_one = LLMChain(llm=llm, prompt=first_prompt)

# 提示模板 2 ：接受公司名称，然后输出该公司的长为20个单词的描述
second_prompt = ChatPromptTemplate.from_template(   
    "写一个20字的描述对于下面这个\
    公司：{company_name}的"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True)
product = "大号床单套装"
overall_simple_chain.run(product)




> Entering new SimpleSequentialChain chain...
为一家制造大号床单套装的公司起一个既有吸引力又能体现产品特色的名称，可以考虑以下几个要素：产品的核心价值、目标客户群体、品牌定位以及是否易于记忆和传播。这里提供几个建议：

1. **舒适梦享**（ComfortDreams）：这个名字强调了床单套装带来的舒适体验，适合追求高品质睡眠体验的消费者。

2. **大号优选**（KingSizeSelect）：直接点明了产品规格，同时也传达出选择大号床单套装时的专业性和品质保证。

3. **安眠之选**（SleepEase）：简洁易记，同时强调了床单套装能够帮助消费者获得更好的睡眠质量。

4. **云端睡眠**（CloudSleep）：利用“云端”这一形象比喻，暗示产品如同云朵般柔软，带给用户极致舒适的睡眠体验。

5. **宽适空间**（WideComfort）：强调大号床单套装能适应更大尺寸的床铺，提供更加宽敞舒适的使用感受。

6. **奢华寝居**（LuxuryNest）：适合面向追求生活品质和奢华感的高端市场，强调产品的奢华与舒适性。

7. **夜色柔梦**（NightSoft）：以“夜色”和“柔梦”为灵感，营造出夜晚宁静、舒适的氛围，适合追求温馨睡眠环境的消费者。

在选择名称时，建议进行商标查询，确保所选名称未被他人注册，同时考虑未来的扩展性，以及在不同市场和文化背景下的接受度。
"丝柔巨梦"（SilkSoftGiantDreams）：结合了丝绸般的质感与大尺寸的舒适，既体现了产品的核心价值——极致柔软与大号规格的独特性，又易于记忆和传播，适合广泛的目标客户群体，无论是追求高品质睡眠的个人还是寻求独特礼物的消费者。

> Finished chain.


'"丝柔巨梦"（SilkSoftGiantDreams）：结合了丝绸般的质感与大尺寸的舒适，既体现了产品的核心价值——极致柔软与大号规格的独特性，又易于记忆和传播，适合广泛的目标客户群体，无论是追求高品质睡眠的个人还是寻求独特礼物的消费者。'

### SequentialChain 
当只有一个输入和一个输出时，简单顺序链（SimpleSequentialChain）即可实现。

当有**多个输入或多个输出**时，我们则需要使用顺序链（SequentialChain）来实现。

SequentialChain会按顺序逐一访问每个chain,将输入input_key并得到output_key.

In [33]:
import pandas as pd
from langchain.chains import SequentialChain
from langchain.chat_models import ChatOpenAI    #导入OpenAI模型
from langchain.prompts import ChatPromptTemplate   #导入聊天提示模板
from langchain.chains import LLMChain    #导入LLM链。

llm = ChatOpenAI(model=model_name,temperature=0.9)


In [30]:
#子链1
# prompt模板 1: 翻译成英语（把下面的review翻译成英语）
first_prompt = ChatPromptTemplate.from_template(
    "把下面的评论review翻译成英文:"
    "\n\n{Review}"
)
# chain 1: 输入：Review    输出：英文的 Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="English_Review")

#子链2
# prompt模板 2: 用一句话总结下面的 review
second_prompt = ChatPromptTemplate.from_template(
    "请你用一句话来总结下面的评论review:"
    "\n\n{English_Review}"
)
# chain 2: 输入：英文的Review   输出：总结
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")


#子链3
# prompt模板 3: 下面review使用的什么语言
third_prompt = ChatPromptTemplate.from_template(
    "下面的评论review使用的什么语言:\n\n{Review}"
)
# chain 3: 输入：Review  输出：语言
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")


#子链4
# prompt模板 4: 使用特定的语言对下面的总结写一个后续回复
fourth_prompt = ChatPromptTemplate.from_template(
    "使用特定的语言对下面的总结写一个后续回复:"
    "\n\n总结: {summary}\n\n语言: {language}"
)
# chain 4: 输入： 总结, 语言    输出： 后续回复
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="followup_message")


In [34]:
#输入：review    
#输出：英文review，总结，后续回复 
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)
review='''
本单元教程涵盖了一系列使用 LangChain 构建语言模型应用的实践，包括处理用户评论、基于文档问答、寻求外部知识等。

强大的 LangChain
通过这一系列案例，我们可以深刻体会到 LangChain 极大简化并加速了语言模型应用开发。过去需要数周才能实现的功能，现在只需极少量的代码即可通过 LangChain 快速构建。LangChain已成为开发大模型应用的有力范式，希望大家拥抱这个强大工具，积极探索更多更广泛的应用场景。

不同组合，更多可能性
LangChain 还可以协助我们做什么呢：基于 CSV 文件回答问题、查询 SQL 数据库、与 API 交互，有很多例子通过 Chain 以及不同的提示（Prompts）和输出解析器（output parsers）组合得以实现。

出发，去探索新世界吧
感谢 LangChain 的贡献者们，你们不断丰富文档和案例，让这一框架更易学易用。如果你还未开始使用 LangChain，现在就打开 Python ，运行pip install LangChain吧，一探这一魔法般工具的无限魅力!
'''
overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'Review': '\n本单元教程涵盖了一系列使用 LangChain 构建语言模型应用的实践，包括处理用户评论、基于文档问答、寻求外部知识等。\n\n强大的 LangChain\n通过这一系列案例，我们可以深刻体会到 LangChain 极大简化并加速了语言模型应用开发。过去需要数周才能实现的功能，现在只需极少量的代码即可通过 LangChain 快速构建。LangChain已成为开发大模型应用的有力范式，希望大家拥抱这个强大工具，积极探索更多更广泛的应用场景。\n\n不同组合，更多可能性\nLangChain 还可以协助我们做什么呢：基于 CSV 文件回答问题、查询 SQL 数据库、与 API 交互，有很多例子通过 Chain 以及不同的提示（Prompts）和输出解析器（output parsers）组合得以实现。\n\n出发，去探索新世界吧\n感谢 LangChain 的贡献者们，你们不断丰富文档和案例，让这一框架更易学易用。如果你还未开始使用 LangChain，现在就打开 Python ，运行pip install LangChain吧，一探这一魔法般工具的无限魅力!\n',
 'English_Review': "The tutorial for this unit covers a series of practical applications using LangChain to build language model solutions, encompassing the handling of user comments, document-based Q&A, and seeking external knowledge.\n\nPowerful LangChain\nThrough these case studies, one can deeply appreciate how LangChain significantly simplifies and accelerates the development of language model applications. Tasks that once required several weeks can now be rapidly constructed with just 

### 路由链
到目前为止，我们已经学习了大语言模型链和顺序链。但是，如果我们想做一些更复杂的事情怎么办？一个相当常见但基本的操作是根据输入将其路由到一条链，具体取决于该输入到底是什么。如果你有多个子链，每个子链都专门用于特定类型的输入，那么可以组成一个路由链，它首先决定将它传递给哪个子链，然后将它传递给那个链。

路由器由两个组件组成：

**路由链（Router Chain）：** 负责选择要调用的下一个链
**destination_chains：** 路由器链可以路由到的链


### 使用方法
一般使用路由链时，有固定的几个步骤：
- 准备多个链的Prompt提示词，然后各自封装成链。
- 将可能路由到的链封装到destination_chains里。
- 构建多提示词和RouterChain ，负责选择下一个要调用的链。
- 构建默认链。
- 使用MultiPromptChain选择某个链，然后再去执行此链。


In [38]:
from langchain.chains.router import MultiPromptChain  #导入多提示链
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate
llm = ChatOpenAI(model=model_name,temperature=0)


In [39]:
# 中文
#第一个提示适合回答物理问题
physics_template = """你是一个非常聪明的物理专家。 \
你擅长用一种简洁并且易于理解的方式去回答问题。\
当你不知道问题的答案时，你承认\
你不知道.

这是一个问题:
{input}"""


#第二个提示适合回答数学问题
math_template = """你是一个非常优秀的数学家。 \
你擅长回答数学问题。 \
你之所以如此优秀， \
是因为你能够将棘手的问题分解为组成部分，\
回答组成部分，然后将它们组合在一起，回答更广泛的问题。

这是一个问题：
{input}"""


#第三个适合回答历史问题
history_template = """你是以为非常优秀的历史学家。 \
你对一系列历史时期的人物、事件和背景有着极好的学识和理解\
你有能力思考、反思、辩证、讨论和评估过去。\
你尊重历史证据，并有能力利用它来支持你的解释和判断。

这是一个问题:
{input}"""


#第四个适合回答计算机问题
computerscience_template = """ 你是一个成功的计算机科学专家。\
你有创造力、协作精神、\
前瞻性思维、自信、解决问题的能力、\
对理论和算法的理解以及出色的沟通技巧。\
你非常擅长回答编程问题。\
你之所以如此优秀，是因为你知道  \
如何通过以机器可以轻松解释的命令式步骤描述解决方案来解决问题，\
并且你知道如何选择在时间复杂性和空间复杂性之间取得良好平衡的解决方案。

这还是一个输入：
{input}"""

# 中文
prompt_infos = [
    {
        "名字": "物理学", 
        "描述": "擅长回答关于物理学的问题", 
        "提示模板": physics_template
    },
    {
        "名字": "数学", 
        "描述": "擅长回答数学问题", 
        "提示模板": math_template
    },
    {
        "名字": "历史", 
        "描述": "擅长回答历史问题", 
        "提示模板": history_template
    },
    {
        "名字": "计算机科学", 
        "描述": "擅长回答计算机科学问题", 
        "提示模板": computerscience_template
    }
]


In [40]:
## 准备多个路由并封装成链
## 构造destination_chains
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["名字"]
    prompt_template = p_info["提示模板"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['名字']}: {p['描述']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
destinations_str

'物理学: 擅长回答关于物理学的问题\n数学: 擅长回答数学问题\n历史: 擅长回答历史问题\n计算机科学: 擅长回答计算机科学问题'

In [41]:
## 设置默认路由
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [43]:
# 多提示路由模板
MULTI_PROMPT_ROUTER_TEMPLATE = """给语言模型一个原始文本输入，\
让其选择最适合输入的模型提示。\
系统将为您提供可用提示的名称以及最适合改提示的描述。\
如果你认为修改原始输入最终会导致语言模型做出更好的响应，\
你也可以修改原始输入。


<< 格式 >>
返回一个带有JSON对象的markdown代码片段，该JSON对象的格式如下：
```json
{{{{
    "destination": 字符串 \ 使用的提示名字或者使用 "DEFAULT"
    "next_inputs": 字符串 \ 原始输入的改进版本
}}}}



记住：“destination”必须是下面指定的候选提示名称之一，\
或者如果输入不太适合任何候选提示，\
则可以是 “DEFAULT” 。
记住：如果您认为不需要任何修改，\
则 “next_inputs” 可以只是原始输入。

<< 候选提示 >>
{destinations}

<< 输入 >>
{{input}}

<< 输出 (记得要包含 ```json)>>

样例:
<< 输入 >>
"什么是黑体辐射?"
<< 输出 >>
```json
{{{{
    "destination": 字符串 \ 使用的提示名字或者使用 "DEFAULT"
    "next_inputs": 字符串 \ 原始输入的改进版本
}}}}

"""


In [44]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

#使用MultiPromptChain选择某个链，然后再去执行此链
chain = MultiPromptChain(router_chain=router_chain,    #l路由链路
                         destination_chains=destination_chains,   #目标链路
                         default_chain=default_chain,      #默认链路
                         verbose=True   
                        )


In [45]:
chain.run("什么是黑体辐射？")




> Entering new MultiPromptChain chain...
物理学: {'input': '什么是黑体辐射？'}
> Finished chain.


'黑体辐射是指理想化的物体（称为黑体）在任何温度下对所有波长的电磁辐射的完全吸收和发射的能力。这种现象在物理学中非常重要，因为它揭示了能量如何以量子的形式进行离散化，并且是量子力学和相对论发展的关键驱动力之一。\n\n在现实世界中，没有真正的黑体存在，但许多实际物体可以近似为黑体，特别是当它们的表面非常光滑且温度足够高时。例如，在极高温度下的恒星表面或加热至极高温度的金属块可以被视为黑体的近似。\n\n黑体辐射的强度与波长和温度有关，这一关系通过维恩位移定律和斯蒂芬-玻尔兹曼定律来描述。其中，维恩位移定律表明最大辐射强度对应的波长与温度成反比，而斯蒂芬-玻尔兹曼定律则指出黑体的总辐射功率与温度的四次方成正比。\n\n黑体辐射理论的发现者是德国物理学家威廉·维恩、德国物理学家罗伯特·斯特藩和德国物理学家马克斯·普朗克，他们因此获得了诺贝尔物理学奖。普朗克为了解释黑体辐射的实验结果，提出了能量量子化的概念，这是量子力学诞生的基石之一。'

In [46]:
chain.run("2+2等于多少？")



> Entering new MultiPromptChain chain...
数学: {'input': '2+2等于多少？'}
> Finished chain.


'2 + 2 等于 4。'

In [47]:
chain.run("为什么我们身体里的每个细胞都包含DNA？")



> Entering new MultiPromptChain chain...
物理学: {'input': '为什么我们身体里的每个细胞都包含DNA？'}
> Finished chain.


'我们的身体由许多不同类型的细胞组成，每一个细胞都包含DNA的原因在于DNA是遗传信息的携带者。DNA（脱氧核糖核酸）分子包含了指导细胞生长、分裂和执行特定功能的指令。这些指令通过基因的形式编码在DNA序列中。\n\n当一个新细胞从现有的细胞中分裂出来时，它需要复制其所有遗传物质以便于新细胞也能正常发育和功能。因此，DNA必须在细胞分裂前被精确地复制，确保两个新细胞都拥有完整的遗传信息。这样，无论是制造蛋白质、调节生理过程还是执行特定功能，细胞都能按照预定的蓝图进行工作。\n\n简而言之，每个细胞都包含DNA是因为DNA携带着构建和维持生命所必需的遗传信息，而这个信息需要在细胞分裂时得到复制和传递给新生细胞。'